In [1]:
# imports for model
import keras_hub
import pathlib
import random

import keras
from keras import ops
from keras_nlp.tokenizers import WordPieceTokenizer

import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_string

c:\adriana\official-code\2025-ASL-data\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imports for pre-parsing

from pyparsing import Word, alphas as pp_alpha, nums as pp_nums
import pyparsing as pp
import re

In [3]:
# handshapes

handshapelist = [
  '1',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '25',
  'A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H-U',
  'I',
  'K',
  'L',
  'M',
  'N',
  'O',
  'R',
  'S',
  'T',
  'U',
  'V',
  'W',
  'X',
  'Y',
  'C-L',
  'U-L',
  'B-L',
  'P-K',
  'Q-G',
  'L-X',
  'I-L-Y',
  '5-C',
  '5-C-L',
  '5-C-tt',
  'alt-M',
  'alt-N',
  'alt-P',
  'B-xd',
  'baby-O',
  'bent-1',
  'bent-B',
  'bent-B-L',
  'bent-horns',
  'bent-L',
  'bent-M',
  'bent-N',
  'bent-U',
  'cocked-8',
  'cocked-F',
  'cocked-S',
  'crvd-5',
  'crvd-B',
  'crvd-flat-B',
  'crvd-L',
  'crvd-sprd-B',
  'crvd-U',
  'crvd-V',
  'fanned-flat-O',
  'flat-B',
  'flat-C',
  'flat-F',
  'flat-G',
  'flat-O-2',
  'flat-O',
  'full-M',
  'horns',
  'loose-E',
  'O-2-horns',
  'open-8',
  'open-F',
  'sml-C-3',
  'tight-C-2',
  'tight-C',
  'X-over-thumb'
]

In [4]:
# regex rules

number_regexp = r"[0-9]+(?:\.[0-9]+)?s?"
alpha_regexp = r"(?!((?:THUMB-)?(?:IX-|POSS-|SELF-)))[A-Z0-9](?:[A-Z0-9/'-]*[A-Z0-9])?(?:\.|:[0-9])?"
lookahead_regexp = r"(?:(?![a-z])|(?=wg))"

word_all_regexp = r"""(?x)
    (?: %s | %s )
    %s
""" % (number_regexp, alpha_regexp, lookahead_regexp)

In [5]:
# conventions kept for parsing

cl_prefix = pp.one_of(["CL", "DCL", "LCL", "SCL", "BCL", "BPCL", "PCL", "ICL"])
ns_prefix = pp.Literal("ns")
fs_prefix = pp.Literal("fs")
lex_exceptions = pp.one_of(["part", "WHAT"])
aspect_text = pp.Literal("aspect")
index_core_ix = pp.Literal("IX")
other_index_core = pp.one_of(["POSS", "SELF"])
handshape = pp.one_of(handshapelist)
person = pp.one_of(["1p", "2p", "3p"])
arc = pp.Literal("arc") 
loc = pp.Literal("loc")
pl = pp.Literal("pl")
compound = pp.Literal("+")
hashtag = pp.Literal("#")
sym = pp.Literal(">")
par1 = pp.Literal("(")
par2 = pp.Literal(")")
dash = pp.Literal("-")
contraction = pp.Literal("^")
colon = pp.Literal(":")
omit_quote = pp.Literal("xx")
period = pp.Literal(".")
alpha = Word(pp_alpha, max=1)
num = Word(pp_nums, max=1)
word = pp.Regex(word_all_regexp, flags=re.X)

In [6]:
# grammar rules

full_grammar = pp.OneOrMore(
    cl_prefix |               # classifiers like CL, DCL, etc.
    word |
    ns_prefix |               # non-specific ns
    fs_prefix |               # fingerspelling fs
    index_core_ix |           # IX
    other_index_core |        # POSS, SELF
    person |                  # 1p, 2p, 3p
    lex_exceptions |          # part, WHAT
    aspect_text |             # aspect
    arc |                     # arc
    loc |                     # loc
    pl |                      # pl
    handshape |               # handshapes like B, 1, 5, etc.
    compound |                # +
    hashtag |                 # #
    sym |                     # >
    contraction |             # ^
    colon |                   # :
    par1 | par2 |             # ( and )
    omit_quote |              # xx
    period |                  # .
    dash |
    num |                    # numbers last resort
    alpha                     # fallback LAST
)

In [7]:
# testing grammar parsing

trial = full_grammar.parse_string("SCL:1xx", parse_all=True).asList()
trial2 = full_grammar.parse_string("IX-1p BCLxx FIND/FIND-OUT fs-HER", parse_all=True).asList()

print(trial)
print(trial2)

['SCL', ':', '1', 'xx']
['IX', '-', '1p', 'BCL', 'xx', 'FIND/FIND-OUT', 'fs', '-', 'HER']


In [8]:
# tokenize based on predefined grammar rules

def custom_asl_tokenize(text):
    try:
        return full_grammar.parse_string(text, parse_all=True).asList()
    except pp.ParseException as pe:
        print(f"Failed to parse: {pe}")
        return []

In [9]:
def custom_eng_tokenize(text):
    # Add spaces around punctuation (preserving it)
    text = re.sub(r'([^\w\s])', r' \1 ', text)
    # Convert to lowercase
    text = text.lower()
    # Split on whitespace
    tokens = text.split()
    return tokens

In [10]:
# testing custom_asl_tokenize

trial = custom_asl_tokenize("SCL:1xx")
trial2 = custom_asl_tokenize('SCL:1xx SHOWER WASH FEEL THUMBS-UP/GOOD')

print(trial)
print(trial2)

['SCL', ':', '1', 'xx']
['SCL', ':', '1', 'xx', 'SHOWER', 'WASH', 'FEEL', 'THUMBS-UP/GOOD']


In [11]:
# model parameters / hyperparameters

BATCH_SIZE = 64
EPOCHS = 30  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 50
ENG_VOCAB_SIZE = 5056
ASL_VOCAB_SIZE = 2283
num_samples = 1400

EMBED_DIM = 128
INTERMEDIATE_DIM =1024
NUM_HEADS = 8
data_path = "C:\\adriana\\official-code\\2025-ASL-data\\data\\sent_pairs.txt"

In [12]:
# generate
    # 1) list of eng-asl sentence pairs
    # 2) set of unique english vocab
    # 3) set of unique asl vocab

text_pairs = []
eng_tokens = set()
asl_tokens = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
 
for line in lines[: min(num_samples, len(lines) - 1)]:
    pair = []
    eng_text, asl_text = line.split("\t")
    pair.append(eng_text)
    pair.append(asl_text)
    text_pairs.append(pair)

for pair in text_pairs:
    sent_tokens = custom_eng_tokenize(pair[0])
    for token in sent_tokens:
        if token not in eng_tokens:
            eng_tokens.add(token)
            
for pair in text_pairs:
    sent_tokens = custom_asl_tokenize(pair[1])
    for token in sent_tokens:
        if token not in asl_tokens:
            asl_tokens.add(token)

eng_tokens = sorted(list(eng_tokens))
asl_tokens = sorted(list(asl_tokens))

print("eng_tokens:", eng_tokens)
print("output_tokens", asl_tokens)
num_encoder_tokens = len(eng_tokens)
num_decoder_tokens = len(asl_tokens)
print("num_eng_tokens", num_encoder_tokens)
print("num_asl_tokens", num_decoder_tokens)
print(asl_tokens)

Failed to parse: Expected end of text, found '/'  (at char 19), (line:1, col:20)
Failed to parse: Expected end of text, found '/'  (at char 23), (line:1, col:24)
eng_tokens: ['!', '"', '$', "'", '(', ')', '*', ',', '-', '.', '/', '1', '10', '100', '11', '110', '150', '3', '30', '300', '350', '4', '45', '5', '50', '500', '50s', '60s', '7', '70', '75', '8', '80', '85', '88', '9', '90', '96', ':', ';', '?', 'a', 'abdominal', 'able', 'about', 'accepted', 'accident', 'across', 'act', 'action', 'actually', 'admit', 'advantage', 'adventure', 'advice', 'after', 'again', 'against', 'ago', 'agree', 'ah', 'ahead', 'ahh', 'ail', 'air', 'ali', 'all', 'alone', 'along', 'already', 'alright', 'also', 'am', 'amazing', 'an', 'and', 'angeles', 'angled', 'angry', 'animal', 'ankle', 'ann', 'another', 'answer', 'any', 'anymore', 'anyone', 'anything', 'anyway', 'anywhere', 'applaud', 'apple', 'approach', 'are', 'area', 'arm', 'arms', 'around', 'around12', 'arrest', 'arrested', 'arrive', 'arrived', 'arrives',

In [13]:
# glimpse pairs

for _ in range(5):
    print(random.choice(text_pairs))

["But in the LA area, they didn't see that way.", 'BUT IN ns-#LA AREA 5xx IX-3p NOT SEE THAT fs-WAY']
['For how long has that student been reading the book?', 'IX-3p STUDENT IX-3p READ BOOK UP-TO-NOW part']
['It is empty.', 'ALL-GONE']
["Maybe the baby will be hearing, but we'll still love it the same. ", 'MAYBE HEARING IX-1p STILL LOVE SAME 5xx']
['You have to make a name for yourself before you join the "elite" clubs.', 'GROUP/TOGETHER-arc IX-2p MUST MAKE POSS-2p NAME BEFORE JOIN fs-CLUB GROUP/TOGETHER fs-E xx NAME fs-ELITE fs-CLUB SOCIAL/INTERACT']


In [14]:
# split data

random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

1400 total pairs
980 training pairs
210 validation pairs
210 test pairs


In [15]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_vocab = eng_tokens + reserved_tokens
asl_vocab = asl_tokens + reserved_tokens

print(eng_vocab)
print(asl_vocab)

['!', '"', '$', "'", '(', ')', '*', ',', '-', '.', '/', '1', '10', '100', '11', '110', '150', '3', '30', '300', '350', '4', '45', '5', '50', '500', '50s', '60s', '7', '70', '75', '8', '80', '85', '88', '9', '90', '96', ':', ';', '?', 'a', 'abdominal', 'able', 'about', 'accepted', 'accident', 'across', 'act', 'action', 'actually', 'admit', 'advantage', 'adventure', 'advice', 'after', 'again', 'against', 'ago', 'agree', 'ah', 'ahead', 'ahh', 'ail', 'air', 'ali', 'all', 'alone', 'along', 'already', 'alright', 'also', 'am', 'amazing', 'an', 'and', 'angeles', 'angled', 'angry', 'animal', 'ankle', 'ann', 'another', 'answer', 'any', 'anymore', 'anyone', 'anything', 'anyway', 'anywhere', 'applaud', 'apple', 'approach', 'are', 'area', 'arm', 'arms', 'around', 'around12', 'arrest', 'arrested', 'arrive', 'arrived', 'arrives', 'arriving', 'as', 'aside', 'ask', 'asked', 'asks', 'assistant', 'at', 'ate', 'attack', 'attacker', 'attention', 'average', 'awake', 'away', 'awful', 'b', 'babies', 'baby', '

In [16]:
print("English Tokens: ", eng_vocab[100:110])
print("ASL Tokens: ", asl_vocab[100:110])

English Tokens:  ['arrested', 'arrive', 'arrived', 'arrives', 'arriving', 'as', 'aside', 'ask', 'asked', 'asks']
ASL Tokens:  ['BED', 'BEER', 'BEFORE', 'BELIEVE', 'BELONG', 'BELT', 'BEST', 'BETTER', 'BETWEEN', 'BEVERLY']


In [17]:
eng_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)
asl_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=asl_vocab, lowercase=False, split=False
)

ImportError: WordPieceTokenizer requires `tensorflow` and `tensorflow-text` for text processing. Run `pip install tensorflow-text` to install both packages or visit https://www.tensorflow.org/install

If `tensorflow-text` is already installed, try importing it in a clean python session. Your installation may have errors.

KerasHub uses `tf.data` and `tensorflow-text` to preprocess text on all Keras backends. If you are running on Jax or Torch, this installation does not need GPU support.

In [ ]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    eng_tokenizer.detokenize(eng_tokens_ex),
)

print()

asl_input_ex = text_pairs[0][1]
print(f"ASL input: '{asl_input_ex}'") 
asl_tokens_ex = asl_tokenizer.tokenize(asl_input_ex)
print("ASL Gloss sentence: ", asl_input_ex)
print("Tokens: ", asl_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    asl_tokenizer.detokenize(asl_tokens_ex),
)

In [ ]:
def preprocess_batch(eng, asl):
    batch_size = ops.shape(asl)[0]

    eng = eng_tokenizer(eng)
    asl = asl_tokenizer(asl)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.
    eng_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Add special tokens (`"[START]"` and `"[END]"`) to `asl` and pad it as well.
    asl_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=asl_tokenizer.token_to_id("[START]"),
        end_value=asl_tokenizer.token_to_id("[END]"),
        pad_value=asl_tokenizer.token_to_id("[PAD]"),
    )
    asl = asl_start_end_packer(asl)

    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": asl[:, :-1],
        },
        asl[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, asl_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    asl_texts = list(asl_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, asl_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

In [ ]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_hub.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=ASL_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_hub.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(ASL_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

In [ ]:
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

In [ ]:
def decode_sequences(input_sentences):
    batch_size = 1

    # Tokenize the encoder input.
    encoder_input_tokens = ops.convert_to_tensor(eng_tokenizer(input_sentences))
    if len(encoder_input_tokens[0]) < MAX_SEQUENCE_LENGTH:
        pads = ops.full((1, MAX_SEQUENCE_LENGTH - len(encoder_input_tokens[0])), 0)
        encoder_input_tokens = ops.concatenate(
            [encoder_input_tokens.to_tensor(), pads], 1
        )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        # Ignore hidden states for now; only needed for contrastive search.
        hidden_states = None
        return logits, hidden_states, cache

    # Build a prompt of length 40 with a start token and padding tokens.
    length = 40
    start = ops.full((batch_size, 1), asl_tokenizer.token_to_id("[START]"))
    pad = ops.full((batch_size, length - 1), asl_tokenizer.token_to_id("[PAD]"))
    prompt = ops.concatenate((start, pad), axis=-1)

    generated_tokens = keras_hub.samplers.GreedySampler()(
        next,
        prompt,
        stop_token_ids=[asl_tokenizer.token_to_id("[END]")],
        index=1,  # Start sampling after start token.
    )
    generated_sentences = asl_tokenizer.detokenize(generated_tokens)
    return generated_sentences


test_eng_texts = [pair[0] for pair in test_pairs]
for i in range(2):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequences([input_sentence])
    translated = translated.numpy()[0].decode("utf-8")
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    print(f"** Example {i} **")
    print(input_sentence)
    print(translated)
    print()

In [ ]:
rouge_1 = keras_hub.metrics.RougeN(order=1)
rouge_2 = keras_hub.metrics.RougeN(order=2)

for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    translated_sentence = decode_sequences([input_sentence])
    translated_sentence = translated_sentence.numpy()[0].decode("utf-8")
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )

    rouge_1(reference_sentence, translated_sentence)
    rouge_2(reference_sentence, translated_sentence)

print("ROUGE-1 Score: ", rouge_1.result())
print("ROUGE-2 Score: ", rouge_2.result())